In [41]:
import json
import pandas as pd
import numpy as np
from functools import reduce

In [42]:
def is_num(x):
    try:
        float(x)
        return True
    except:
        return False

In [43]:
f = open('./data_struct_info/data_struct.json')
data_struct = json.load(f)

In [44]:
data_struct

{'exam_result_list.xlsx': {'column2pos': {'exam_id': 0,
   'exam_result_number': 1,
   'exam_result_string': 2},
  'columns': ['exam_id', 'exam_result_number', 'exam_result_string']},
 'student_exam_list.xlsx': {'column2pos': {'student_id': 0,
   'exam_id': 1,
   'exam_date': 2,
   'exam_type': 3},
  'columns': ['student_id', 'exam_id', 'exam_date', 'exam_type']},
 'student_exam_result_list.xlsx': {'column2pos': {'student_id': 0,
   'exam_id': 1,
   'exam_date': 2,
   'exam_type': 3,
   'exam_result_number': 4,
   'exam_result_string': 5},
  'columns': ['student_id',
   'exam_id',
   'exam_date',
   'exam_type',
   'exam_result_number',
   'exam_result_string']}}

In [45]:
f = open('./input_data/example.json')
jy_json = json.load(f)
jy_json 

{'0870': {'english': [{'时间': '20190406', '结果': 97},
   {'时间': '20190409', '结果': 'A+'}]},
 '0871': {'chinese': [{'时间': '20180406', '结果': 32}],
  'chemistry': [{'时间': '20160409', '结果': 'G'}]}}

In [46]:
ID = 60000
def append(arr, x):
    arr.append(x)
    return arr
def f(ac, x):
    user_id, type_res_obj = x
    def g(ac, x):
        t, res_arr = x
        reduce(lambda acc, y:append(acc, {
            **y,
            'type':t,
            'user_id':user_id
        }), res_arr,ac)
        return ac
    reduce(lambda acc,y:g(acc,y),type_res_obj.items(), ac )
    return ac
jy_list_temp = reduce(f, jy_json.items(), [])
jy_list_temp # Fold outward list, add its ancestor

[{'时间': '20190406', '结果': 97, 'type': 'english', 'user_id': '0870'},
 {'时间': '20190409', '结果': 'A+', 'type': 'english', 'user_id': '0870'},
 {'时间': '20180406', '结果': 32, 'type': 'chinese', 'user_id': '0871'},
 {'时间': '20160409', '结果': 'G', 'type': 'chemistry', 'user_id': '0871'}]

In [47]:
def f(x):
    offset, row = x
    return {
        **row, 
        'id':offset+ID
    }
jy_list = list(map(f, enumerate(jy_list_temp) ))
# list(jy_json.items())[0]
jy_list # add id, split each item into two dataframe in following

[{'时间': '20190406',
  '结果': 97,
  'type': 'english',
  'user_id': '0870',
  'id': 60000},
 {'时间': '20190409',
  '结果': 'A+',
  'type': 'english',
  'user_id': '0870',
  'id': 60001},
 {'时间': '20180406',
  '结果': 32,
  'type': 'chinese',
  'user_id': '0871',
  'id': 60002},
 {'时间': '20160409',
  '结果': 'G',
  'type': 'chemistry',
  'user_id': '0871',
  'id': 60003}]

In [48]:
ex_name = 'exam_result_list.xlsx'
df_struct = data_struct[ex_name]
column2pos = df_struct['column2pos']
print(df_struct)
def f(x):
    row = [np.nan for i in df_struct['columns']]
    row[column2pos['exam_id']] = x['id']
    if is_num(x['结果']):
        row[column2pos['exam_result_number']] = x['结果']
    else:
        row[column2pos['exam_result_string']] = x['结果']
    return row
all_row = list(map(f, jy_list))
df = pd.DataFrame(all_row,columns=data_struct[ex_name]['columns'])
df.to_excel('./output_data/%s'%ex_name)
df

{'column2pos': {'exam_id': 0, 'exam_result_number': 1, 'exam_result_string': 2}, 'columns': ['exam_id', 'exam_result_number', 'exam_result_string']}


,exam_id,exam_result_number,exam_result_string
0,60000,97.0,NaN
1,60001,NaN,A+
2,60002,32.0,NaN
3,60003,NaN,G


In [50]:
ex_name = 'student_exam_list.xlsx'
df_struct = data_struct[ex_name]
column2pos = df_struct['column2pos']
print(df_struct)
def f(x):
    row = [np.nan for i in df_struct['columns']]
    row[column2pos['exam_id']] = x['id']
    row[column2pos['student_id']] = x['user_id']
    row[column2pos['exam_date']] = x['时间']
    row[column2pos['exam_type']] = x['type']
    return row
all_row = list(map(f, jy_list))
df = pd.DataFrame(all_row,columns=data_struct[ex_name]['columns'])
df.to_excel('./output_data/%s'%ex_name)
df

{'column2pos': {'student_id': 0, 'exam_id': 1, 'exam_date': 2, 'exam_type': 3}, 'columns': ['student_id', 'exam_id', 'exam_date', 'exam_type']}


,student_id,exam_id,exam_date,exam_type
0,0870,60000,20190406,english
1,0870,60001,20190409,english
2,0871,60002,20180406,chinese
3,0871,60003,20160409,chemistry
